In [16]:
import numpy as np
from sklearn import metrics
import optuna


def balanced_log_loss(y_true, y_pred):
    nc = np.bincount(y_true)
    return metrics.log_loss(y_true, y_pred, sample_weight=1 / nc[y_true], eps=1e-15)


In [17]:
from tabpfn import TabPFNClassifier

In [18]:
import pandas as pd
from pathlib import Path

path = Path('./data')
output = Path('./output')
output.mkdir(exist_ok=True)

In [19]:
df = pd.read_csv(path/'train.csv', index_col='Id')

In [20]:
from sklearn import pipeline, model_selection, impute

def get_tree_preprocess_pipeline():
    return pipeline.make_pipeline(
        impute.KNNImputer(n_neighbors=10, weights="distance"),
    ).set_output(transform="pandas")


# Load data

df = pd.read_csv(path / "train.csv", index_col="Id")
dep_vars = ["Class"]

drop_vars = ["EJ"]
df.drop(columns=drop_vars, inplace=True)

train_df, test_df = model_selection.train_test_split(df, test_size=0.4, stratify=df[dep_vars], random_state=33)

preprocessor = get_tree_preprocess_pipeline()

# Preprocess training data
X_pre = preprocessor.fit_transform(train_df.drop(columns=dep_vars))
train_df = pd.merge(X_pre, train_df[dep_vars], left_index=True, right_index=True)
X = train_df.drop(columns=dep_vars, errors="ignore")
y = train_df[dep_vars].values.ravel()

# Preprocess test data
X_test_pre = preprocessor.transform(test_df.drop(columns=dep_vars))
test_df = pd.merge(X_test_pre, test_df[dep_vars], left_index=True, right_index=True)
X_test = test_df.drop(columns=dep_vars, errors="ignore")
y_test = test_df[dep_vars].values.ravel()

# Calculate scale_pos_weight
scale_pos_weight = df["Class"].value_counts()[0] / df["Class"].value_counts()[1]

In [21]:
# Resample
from imblearn.over_sampling import SMOTE


def resample(X, y):
    sampler = SMOTE()
    X_res, y_res = sampler.fit_resample(X, y)
    return X_res, y_res

In [22]:
def objective(trial):
    classifier = TabPFNClassifier(
        device="cuda",
        N_ensemble_configurations=trial.suggest_int("N_ensemble_configurations", 4, 128, step=4),
        feature_shift_decoder=trial.suggest_categorical("feature_shift_decoder", [True, False]),
        only_inference=False,
    )
    
    X_res, y_res = resample(X, y)
    
    classifier.fit(X_res, y_res)

    p_eval = classifier.predict_proba(X_test)
    return balanced_log_loss(y_test, p_eval)

In [ ]:
# study = optuna.delete_study(
#     study_name="tabpfn_no_resample",
#     storage="sqlite:////storage/optuna-final.db",
# )

In [24]:
study = optuna.create_study(
    direction="minimize",
    study_name="tabpfn_resample",
    storage="sqlite:////storage/optuna-final.db",
    load_if_exists=True,
)
study.optimize(objective, n_trials=250)

[I 2023-08-07 02:06:16,879] A new study created in RDB with name: tabpfn_resample


!! Warning: GPyTorch must be installed !!
Using style prior: True
Using cuda device
Using a Transformer with 25.82 M parameters


[I 2023-08-07 02:06:18,537] Trial 0 finished with value: 0.47117188353216266 and parameters: {'N_ensemble_configurations': 52, 'feature_shift_decoder': False}. Best is trial 0 with value: 0.47117188353216266.


!! Warning: GPyTorch must be installed !!
Using style prior: True
Using cuda device
Using a Transformer with 25.82 M parameters


[I 2023-08-07 02:06:21,435] Trial 1 finished with value: 0.4980927958382415 and parameters: {'N_ensemble_configurations': 48, 'feature_shift_decoder': True}. Best is trial 0 with value: 0.47117188353216266.


...

[I 2023-08-07 02:12:52,972] Trial 247 finished with value: 0.49258066689595237 and parameters: {'N_ensemble_configurations': 60, 'feature_shift_decoder': False}. Best is trial 65 with value: 0.46034804817416475.


!! Warning: GPyTorch must be installed !!
Using style prior: True
Using cuda device
Using a Transformer with 25.82 M parameters


[I 2023-08-07 02:12:54,027] Trial 248 finished with value: 0.4850113377045087 and parameters: {'N_ensemble_configurations': 60, 'feature_shift_decoder': False}. Best is trial 65 with value: 0.46034804817416475.


!! Warning: GPyTorch must be installed !!
Using style prior: True
Using cuda device
Using a Transformer with 25.82 M parameters


[I 2023-08-07 02:12:55,091] Trial 249 finished with value: 0.4730577844283935 and parameters: {'N_ensemble_configurations': 68, 'feature_shift_decoder': False}. Best is trial 65 with value: 0.46034804817416475.
